In [1]:
import torch

In [55]:
def factorial(n):
    if n==0:
        return 1
    else:
        return n*factorial(n-1)

def take_all_pwrs(vec,pwr):
    #todo: vectorize (kinda)
    combins=torch.combinations(vec, r=pwr, with_replacement=True)
    out=torch.ones(combins.size()[0])
    for i in torch.t(combins):
        out *= i
    return out

class LinearRegKernel():
    parameters= []
    weights=None
    bias=None
    def __init__(self, num_vars):
        self.weights=torch.rand(num_vars, requires_grad=True)
        self.bias=torch.rand(1, requires_grad=True)
        self.parameters=[self.weights,self.bias]
    def forward(self,mtx):
        long_bias=self.bias.repeat([1,mtx.size()[1]])
        return torch.matmul(self.weights,mtx)+long_bias
    
class SigmoidalRegKernel():
    parameters= []
    weights=None
    bias=None
    sigmoid=torch.nn.Sigmoid()
    def __init__(self, num_vars):
        self.weights=torch.rand(num_vars)
        self.bias=torch.rand(1)
        self.parameters=[self.weights,self.bias]
    def forward(self,mtx):
        long_bias=self.bias.repeat([1,mtx.size()[1]])
        return self.sigmoid(torch.matmul(self.weights,mtx)+long_bias)

class LogRegKernel():
    parameters= []
    weights=None
    bias=None
    def __init__(self, num_vars):
        self.weights=torch.rand(num_vars)
        self.bias=torch.rand(1)
        self.parameters=[self.weights,self.bias]
    def forward(self,mtx):
        long_bias=self.bias.repeat([1,mtx.size()[1]])
        return torch.log(torch.matmul(self.weights,mtx)+long_bias)

class ExpRegKernel():
    parameters= []
    weights=None
    bias=None
    def __init__(self, num_vars):
        self.weights=torch.rand(num_vars)
        self.bias=torch.rand(1)
        self.parameters=[self.weights,self.bias]
    def forward(self,mtx):
        long_bias=self.bias.repeat([1,mtx.size()[1]])
        return torch.exp(torch.matmul(self.weights,mtx)+long_bias)

class PolyRegKernel():
    parameters= []
    weights=None
    bias=None
    power=None
    def __init__(self, num_vars, power):
        self.power=power
        num_terms=int(factorial(num_vars+power-1) / factorial(power) / factorial(num_vars-1))
        self.weights=torch.rand(num_terms)
        self.bias=torch.rand(1)
        self.parameters=[self.weights,self.bias]
    def forward(self,mtx):
        #TODO: Vectorize the last part
        cols=[]
        for i in torch.t(mtx):
            cols.append(take_all_pwrs(i,self.power))
        new_mtx=torch.t(torch.stack(cols))
        long_bias=self.bias.repeat([1,mtx.size()[1]])
        return torch.matmul(self.weights,new_mtx)+long_bias

def SGDTrain(kernel, data, ground, loss=torch.nn.MSELoss(), iterations=1000, learning_rate=.1, return_losses=False):
    optim=torch.optim.SGD(kernel.parameters, lr=learning_rate)
    if (return_losses):
        losses=[]
        for i in range(iterations):
            with torch.set_grad_enabled(True):
                pred=kernel.forward(data)
                ls=loss(pred,ground)
                losses.append(ls.item())
                ls.backward()
                optim.step()
        return [kernel,losses]
    else:
        for i in range(iterations):
            with torch.set_grad_enabled(True):
                pred=kernel.forward(data)
                ls=loss(pred,ground)
                ls.backward()
                optim.step() 
        return kernel

def CustomTrain(kernel, optim, data, ground, loss=torch.nn.MSELoss(), iterations=1000, return_losses=False):
    if (return_losses):
        losses=[]
        for i in range(iterations):
            with torch.set_grad_enabled(True):
                pred=kernel.forward(data)
                ls=loss(pred,ground)
                losses.append(ls.item())
                ls.backward()
                optim.step()
        return [kernel,losses]
    else:
        for i in range(iterations):
            with torch.set_grad_enabled(True):
                pred=kernel.forward(data)
                ls=loss(pred,ground)
                ls.backward()
                optim.step() 
        return kernel

In [59]:
model=SGDTrain(LinearRegKernel(3),torch.tensor([[1,2],[3,4],[5,6]]).to(torch.float),torch.tensor([[1,2]]).to(torch.float),iterations=10000, learning_rate=.001, return_losses=True)
model[1]

[46.92313766479492,
 38.630210876464844,
 25.048364639282227,
 11.097504615783691,
 1.831160068511963,
 0.6059606671333313,
 7.865695476531982,
 20.980566024780273,
 35.19981002807617,
 45.37260818481445,
 47.813907623291016,
 41.63934326171875,
 29.08551788330078,
 14.699889183044434,
 3.693437099456787,
 0.05307894945144653,
 5.097426891326904,
 16.999135971069336,
 31.446857452392578,
 43.20696258544922,
 48.019371032714844,
 44.1407585144043,
 32.976009368896484,
 18.569347381591797,
 6.139341354370117,
 0.18853749334812164,
 2.872443675994873,
 13.218727111816406,
 27.47943878173828,
 40.48865509033203,
 47.53378677368164,
 46.06269454956055,
 36.60813522338867,
 22.59479522705078,
 9.098745346069336,
 1.008663535118103,
 1.2549662590026855,
 9.74829387664795,
 23.41189193725586,
 37.29610824584961,
 46.37138748168945,
 47.35015106201172,
 39.877723693847656,
 26.660755157470703,
 12.486827850341797,
 2.4901671409606934,
 0.2918170988559723,
 6.687965393066406,
 19.361522674560547